## 2024 Elections Around the World

by Katrina Ventura

2024 has been dubbed the "elections year". Voters from at least 64 countries will be heading to the polls to decide their new leaders. Some of the countries include the influential democracies such as the United States, European Union, Russia, and India. The leaders that will come out of this year's elections will determine the kind of policies that will shape the world for the next 3 to 5 years. 

For my second Data Studio project, the goal is to create a data visualization that will auto-update to reflect the results of each election that will be held.  The graphic will be a diverging bar plot where each row shows the month, and each entry is a country. The bar plot will measure the voter turnout, the negative and positive values will be determined by the political ideology of the elected leader. Data will be collected from the International Foundation for Electoral Systems' Election Guide for 2024
https://www.electionguide.org/electionresults/?years%5B%5D=2024

Will the world vote left or right?




### Step 1: Access IFES Election Website using Playwright

In [1]:
from playwright.async_api import async_playwright

playwright = await async_playwright().start()
browser = await playwright.chromium.launch(headless=False)


In [2]:
page = await browser.new_page()
await page.goto("https://www.electionguide.org")

<Response url='https://www.electionguide.org/' request=<Request url='https://www.electionguide.org/' method='GET'>>

In [3]:
# Click the "Elections" dropdown element
await page.click('a.nav-link.dropdown-toggle[href="#"][id="navbarDropdown"]')


In [4]:
# Click on the "Advanced Search" link within the dropdown menu
await page.click('a.dropdown-item[href="/election-search/"]')

In [5]:
await page.select_option('select[name="years[]"]', '2024')
await page.click('button[type="submit"].btn.btn-primary')

In [6]:
# Click on the sorting button to sort the date column in ascending order
await page.click('th.sorting[aria-label="Date: activate to sort column ascending"]')


In [7]:
 # Click on the drop-down button
await page.click("select[name='electionsTable_length']")
    
# Choose the "100" option
await page.select_option("select[name='electionsTable_length']", "100")
    

['100']

### Step 2: Scrape Election Guide Results using BeautifulSoup

In [8]:
import requests

from bs4 import BeautifulSoup
html = await page.content()
doc = BeautifulSoup(html)

import pandas as pd


In [9]:
#Select Elections table
table_body = doc.find('table', {'id': 'electionsTable'}).find('tbody')


In [10]:
# Iterate over each row in the table body skipping the first column [0] which contains an image of country flags
data = {
    'Country': [],
    'Election for': [],
    'Date': [],
    'Status': [],
    'Registered': [],
    'Actual Votes': [],
    'Participation': []
}

# Iterate over each row in the table body
for row in table_body.find_all('tr'):
    cells = row.find_all('td')
    
    data['Country'].append(cells[1].text.strip())
    data['Election for'].append(cells[2].text.strip())
    data['Date'].append(cells[3].text.strip())
    data['Status'].append(cells[4].text.strip())
    data['Registered'].append(cells[5].text.strip())
    data['Actual Votes'].append(cells[6].text.strip())
    data['Participation'].append(cells[7].text.strip())

# Create a DataFrame from the collected data
df = pd.DataFrame(data)
df

,Country,Election for,Date,Status,Registered,Actual Votes,Participation
0,Bangladesh,Bangladeshi National Parliament,2024-01-07 (d),Held,"119,691,633",n/a,
1,Bhutan,Bhutanese National Assembly,2024-01-09 (d),Held,"498,135","326,775 (c)",65.6\n %
2,Sint Maarten,Sint Maarten Parliament,2024-01-11 (d),Held,"22,553","14,443 (v)",65.5\n %
3,Taiwan,Taiwanese Legislative Yuan,2024-01-13 (d),Held,"19,566,007","13,776,736 (v)",71.8\n %
4,Taiwan,Taiwanese Presidency,2024-01-13 (d),Held,"19,548,531","13,947,506 (v)",71.9\n %
...,...,...,...,...,...,...,...
95,Senegal,Senegalese Presidency,2024-12-15 (d),Postponed,None,n/a,
96,Uzbekistan,Uzbekistani Legislative Chamber,2024-12-31 (t),Date not confirmed,None,n/a,
97,South Sudan,South Sudanese Transitional National Legislati...,2024-12-31 (t),Date not confirmed,None,n/a,
98,Algeria,Algerian Presidency,2024-12-31 (t),Date not confirmed,None,n/a,


In [11]:
csv_filepath = 'data.csv'
df.to_csv(csv_filepath, index=False)
